# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/workspace/home


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
try: 
    cluster = Cluster(['127.0.0.1']) #If you have a locally installed Apache Cassandra instance
    session = cluster.connect()
except Exception as e:
    print(e)

#### Create Keyspace

In [6]:
# TO-DO: Create a Keyspace

try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity_project
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
# TO-DO: Set KEYSPACE to the keyspace specified above

try:
    session.execute("USE udacity_project")
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [8]:
df = pd.read_csv('event_datafile_new.csv')

In [9]:
df.head(5)

,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
0,Explosions In The Sky,Adelyn,F,0,Jordan,497.47546,free,"Chicago-Naperville-Elgin, IL-IN-WI",458,Your Hand In Mine,7
1,Paul Van Dyk Featuring Jessica Sutta,Layla,F,0,Griffin,425.66485,paid,"Lake Havasu City-Kingman, AZ",672,White Lies (Dave Spoon Remix),24
2,Tim Hughes,Layla,F,1,Griffin,323.47383,paid,"Lake Havasu City-Kingman, AZ",672,God of Justice,24
3,David Cassidy & The Partridge Family,Layla,F,2,Griffin,227.73506,paid,"Lake Havasu City-Kingman, AZ",672,I'll Meet You Halfway,24
4,Snow Patrol,Layla,F,3,Griffin,200.93342,paid,"Lake Havasu City-Kingman, AZ",672,Crack The Shutters,24


In [10]:
## Considerando a denormalização, 1 query = 1 tqbela
## TO-DO: Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4

try:
    query = "CREATE TABLE IF NOT EXISTS music_sessions "
    query += "(artist text, song text, length float, sessionId int, itemInSession int, PRIMARY KEY (sessionId, itemInSession))"
    session.execute(query)
except Exception as e:
    print(e)

In [11]:
nomes_colunas = df.columns

# Iterando para imprimir o nome de cada coluna
for indice, nome_coluna in enumerate(nomes_colunas):
    print(f"Nome da coluna: {nome_coluna}, line[{indice}]")

Nome da coluna: artist, line[0]
Nome da coluna: firstName, line[1]
Nome da coluna: gender, line[2]
Nome da coluna: itemInSession, line[3]
Nome da coluna: lastName, line[4]
Nome da coluna: length, line[5]
Nome da coluna: level, line[6]
Nome da coluna: location, line[7]
Nome da coluna: sessionId, line[8]
Nome da coluna: song, line[9]
Nome da coluna: userId, line[10]


In [12]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## TO-DO: Assign the INSERT statements into the `query` variable
        query = "INSERT INTO music_sessions (artist, song, length, sessionId, itemInSession)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (line[0], line[9], float(line[5]), int(line[8]), int(line[3])))

#### Do a SELECT to verify that the data have been inserted into each table

In [13]:
## TO-DO: Add in the SELECT statement to verify the data was entered into the table

try:
    # Executar a consulta SELECT
    query = "SELECT * FROM music_sessions WHERE sessionid = 338 AND itemInSession = 4"  # Substitua 'music_sessions' pelo nome da sua tabela
    rows = session.execute(query)

    # Iterar sobre as linhas e imprimir resultados
    for row in rows:
        print(
            f"{row.artist.ljust(30)}{row.song.ljust(40)}{str(row.length).ljust(20)}{str(row.sessionid).ljust(5)}{str(row.iteminsession)}"
        )
except Exception as e:
    print(f"Ocorreu um erro: {e}")

Faithless                     Music Matters (Mark Knight Dub)         495.30731201171875  338  4


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [14]:
nomes_colunas = df.columns

# Iterando para imprimir o nome de cada coluna
for indice, nome_coluna in enumerate(nomes_colunas):
    print(f"Nome da coluna: {nome_coluna}, line[{indice}]")

Nome da coluna: artist, line[0]
Nome da coluna: firstName, line[1]
Nome da coluna: gender, line[2]
Nome da coluna: itemInSession, line[3]
Nome da coluna: lastName, line[4]
Nome da coluna: length, line[5]
Nome da coluna: level, line[6]
Nome da coluna: location, line[7]
Nome da coluna: sessionId, line[8]
Nome da coluna: song, line[9]
Nome da coluna: userId, line[10]


In [15]:
## TO-DO: Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182

try:
    query = "CREATE TABLE IF NOT EXISTS music_user_sessions "
    query += "(artist text, song text, firstname text, lastname text, userid int, sessionid int, iteminsession int, PRIMARY KEY ((userid, sessionid), iteminsession))"
    session.execute(query)
except Exception as e:
    print(e)

with open(file, encoding='utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)  # Pular cabeçalho
    for line in csvreader:
        query = "INSERT INTO music_user_sessions (artist, song, firstname, lastname, userid, sessionid, iteminsession)"
        query += " VALUES (%s, %s, %s, %s, %s, %s, %s)"  # Adicionei placeholders para os valores
        session.execute(query, (line[0], line[9], line[1], line[4], int(line[10]), int(line[8]), int(line[3])))

In [16]:
try:
    # Executar a consulta SELECT para obter as informações desejadas
    query = "SELECT artist, song, firstname, lastname FROM music_user_sessions WHERE userid = 10 AND sessionid = 182"
    rows = session.execute(query)

    # Imprimir os resultados com formatação específica
    for row in rows:
        print(
            f"{row.artist[:19].ljust(20)}{row.song[:39].ljust(40)}{row.firstname} {row.lastname}"
        )


except Exception as e:
    print(f"Ocorreu um erro: {e}")

Down To The Bone    Keep On Keepin' On                      Sylvie Cruz
Three Drives        Greece 2000                             Sylvie Cruz
Sebastien Tellier   Kilometer                               Sylvie Cruz
Lonnie Gordon       Catch You Baby (Steve Pitron & Max Sann Sylvie Cruz


In [17]:
## TO-DO: Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
try:
    query = "CREATE TABLE IF NOT EXISTS music_history_by_song "
    query += "(song text, firstname text, lastname text, userid int, PRIMARY KEY (song, userid))"
    session.execute(query)
except Exception as e:
    print(e)

with open(file, encoding='utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)  # Pular cabeçalho
    for line in csvreader:
        query = "INSERT INTO music_history_by_song (song, firstname, lastname, userid)"
        query += " VALUES (%s, %s, %s, %s)"
        session.execute(query, (line[9], line[1], line[4], int(line[10])))

In [18]:
try:
    # Executar a consulta SELECT para obter as informações desejadas
    query = "SELECT firstname, lastname \
            FROM music_history_by_song WHERE song = 'All Hands Against His Own'"
    rows = session.execute(query)

    # Imprimir os resultados com formatação específica
    for row in rows:
        print(
            f"{row.firstname} {row.lastname}"
        )


except Exception as e:
    print(f"Ocorreu um erro: {e}")

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [19]:
## TO-DO: Drop the table before closing out the sessions

try:
    session.execute("DROP TABLE IF EXISTS music_history_by_song")
    session.execute("DROP TABLE IF EXISTS music_user_sessions")
    session.execute("DROP TABLE IF EXISTS music_sessions")
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [20]:
session.shutdown()
cluster.shutdown()